<a href="https://colab.research.google.com/github/TanyaDgrChf/savetheworld2-msimcup/blob/main/AImodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Unzips dataset into TrashNet
# dataset from https://github.com/garythung/trashnet
!unzip /content/dataset-resized.zip -d /content/TrashNet

In [ ]:
# Splits images into test, train and validation
import os
import shutil
from sklearn.model_selection import train_test_split

# Specify the path to the extracted dataset
dataset_dir = '/content/TrashNet/dataset-resized'

# Specify the target directories for train, validation, and test splits
train_dir = '/content/train'
valid_dir = '/content/validation'
test_dir = '/content/test'

# Create the target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get the list of class directories in the dataset
class_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

# Iterate through each class directory and split the images
for class_dir in class_dirs:
    class_path = os.path.join(dataset_dir, class_dir)
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Split the images into train, validation, and test sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, valid_images = train_test_split(train_images, test_size=0.2, random_state=42)

    # Create the target directories for each class in the train, validation, and test sets
    train_class_dir = os.path.join(train_dir, class_dir)
    valid_class_dir = os.path.join(valid_dir, class_dir)
    test_class_dir = os.path.join(test_dir, class_dir)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(valid_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move the images to the corresponding class directories
    for image in train_images:
        src_path = os.path.join(class_path, image)
        dst_path = os.path.join(train_class_dir, image)
        shutil.copy(src_path, dst_path)

    for image in valid_images:
        src_path = os.path.join(class_path, image)
        dst_path = os.path.join(valid_class_dir, image)
        shutil.copy(src_path, dst_path)

    for image in test_images:
        src_path = os.path.join(class_path, image)
        dst_path = os.path.join(test_class_dir, image)
        shutil.copy(src_path, dst_path)

In [ ]:
# AI model itself
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Set the paths to the TrashNet dataset
train_data_dir = '/content/train'
valid_data_dir = '/content/validation'
test_data_dir = '/content/test'

# Set the image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_width, img_height, 3)
)

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)

# Save the trained model
model.save('trashnet_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/trashnet_model.h5')

In [ ]:
# Determine which class is which object
import numpy as np
from PIL import Image

class_labels = ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic', 'Trash']

# Load and preprocess the image
image = Image.open('/content/plastic-bag-9659043.jpg')
image = image.resize((224, 224))  # Resize the image to match the input size of the model
image_array = np.array(image) / 255.0  # Normalize the pixel values

# Expand dimensions to match the input shape of the model
input_image = np.expand_dims(image_array, axis=0)

# Make predictions
predictions = model.predict(input_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the predicted class index to the corresponding label
predicted_class = class_labels[predicted_class_index]

print("Predicted class:", predicted_class)